# NSIP Sheep Genetics: Understanding and Using EBVs

This notebook helps you understand and use Estimated Breeding Values (EBVs) to make smarter breeding decisions for your flock.

**What are EBVs?** They are numbers that estimate the genetic merit of an animal for specific traits. Think of them like a report card for genetics -- they tell you what an animal is likely to pass on to its offspring, not just how the animal itself performed.

EBVs are more reliable than raw performance data because they account for environmental factors (feed, weather, management) and family history. A ram raised on excellent pasture might look great, but his EBVs tell you whether his lambs will also perform well regardless of conditions.

## How to Run This Notebook

**Prerequisites:**
1. Complete the `getting-started.ipynb` notebook first.
2. Make sure the NSIP CLI is installed (`nsip --version` should work in your terminal).
3. Run `uv sync --extra data` to install pandas and other dependencies.

**Instructions:**
1. Start Jupyter with `uv run jupyter notebook` or `uv run jupyter lab`.
2. Open this file (`nsip-sheep-genetics.ipynb`).
3. Run each cell in order from top to bottom using **Shift + Enter**.
4. Replace example animal IDs with your own flock IDs where noted.

## Setup

This cell loads all the tools we will use throughout the notebook. Run it first before anything else.

In [ ]:
# Imports
import json
import subprocess

import pandas as pd

# Configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


def nsip(*args):
    """Run an NSIP CLI command and return parsed JSON results."""
    cmd = ["nsip", "--json"] + list(args)
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
        return None
    return json.loads(result.stdout)

## What Are EBVs?

EBVs (Estimated Breeding Values) predict how an animal's genetics will influence its offspring. Here are the key traits for meat sheep like Katahdins:

| Trait | Abbreviation | What It Means | Goal |
|-------|-------------|---------------|------|
| Birth Weight | BWT | How heavy lambs are at birth | **Lower is better** -- reduces birthing difficulty |
| Weaning Weight | WWT | How heavy lambs are at weaning (~60 days) | **Higher is better** -- faster growth |
| Number of Lambs Born | NLB | How many lambs a ewe produces per lambing | **Higher is better** -- more lambs per ewe |
| Maternal Weaning Weight | MWWT | How well a ewe raises her lambs (milk + mothering) | **Higher is better** -- better mothers |

### A Simple Analogy

Imagine you are buying a truck. The sticker says it can tow 10,000 lbs, but you know the dealer tested it going downhill with a tailwind. EBVs are like an independent, standardized towing test -- they strip away the lucky conditions and tell you what the truck (or in our case, the animal) can really do genetically.

### Reading EBV Numbers

- **Positive numbers** mean above average for the breed.
- **Negative numbers** mean below average.
- **Zero** is the breed average.
- For BWT, a *negative* EBV is actually desirable (lighter birth weights = easier lambing).

## Searching for Katahdin Animals

Let us search the NSIP database for Katahdin sheep (breed ID 640). This gives you a broad view of what animals are registered.

In [ ]:
# Search for Katahdin sheep in the NSIP database
raw_search = nsip("search", "--breed-id", "640")

if raw_search:
    search_by_name = pd.DataFrame(raw_search)
    print(f"Found {len(search_by_name)} animals.")
    print()
    print(search_by_name.head(10).to_string(index=False))

You can also narrow your search by name. Let us search for animals with "storm" in their name.

In [ ]:
# Search by name within the Katahdin breed
raw_named_search = nsip(
    "search", "--breed-id", "640", "--name", "storm"
)

if raw_named_search:
    search_by_storm = pd.DataFrame(raw_named_search)
    print(f"Found {len(search_by_storm)} animals named 'storm'.")
    print()
    print(search_by_storm.to_string(index=False))

## Looking Up a Specific Animal

Once you find an animal you are interested in, you can look up its full details -- including all EBV traits and accuracy values.

Replace the animal ID below with one from your own flock or from the search results above.

In [ ]:
# Replace with an actual animal ID from your flock or search results
animal_id = "EXAMPLE_ID"

raw_animal = nsip("animal", "--animal-id", animal_id)

if raw_animal:
    print(f"Animal: {raw_animal.get('name', 'Unknown')}")
    print(f"Breed:  {raw_animal.get('breed', 'Unknown')}")
    print(f"Sex:    {raw_animal.get('sex', 'Unknown')}")
    print()
    print("--- EBV Summary ---")
    print(json.dumps(raw_animal.get('ebvs', {}), indent=2))

### How to Interpret These Results

When you look at an animal's EBVs, ask yourself:

- **BWT**: Is it negative? Good -- that means lighter lambs at birth and fewer birthing problems.
- **WWT**: Is it positive? Good -- lambs from this animal grow faster to weaning.
- **NLB**: Is it positive? Good -- ewes related to this animal tend to have more lambs.
- **MWWT**: Is it positive? Good -- ewes related to this animal are better mothers.
- **Accuracy**: Higher accuracy (closer to 1.0) means the EBV is more reliable. Young animals with few offspring will have lower accuracy.

## Comparing Rams Side-by-Side

When choosing a ram, it helps to compare several candidates at once. This cell looks up three rams and puts their EBVs in a table for easy comparison.

Replace the IDs below with rams you are actually considering.

In [ ]:
# Replace with actual ram IDs from your flock
ram_ids = ["RAM_ID_1", "RAM_ID_2", "RAM_ID_3"]

ram_records = []
for rid in ram_ids:
    raw_ram = nsip("animal", "--animal-id", rid)
    if raw_ram:
        ebvs = raw_ram.get("ebvs", {})
        ram_records.append({
            "Name": raw_ram.get("name", rid),
            "BWT": ebvs.get("bwt", None),
            "WWT": ebvs.get("wwt", None),
            "NLB": ebvs.get("nlb", None),
            "MWWT": ebvs.get("mwwt", None),
        })

Now let us display the comparison table.

In [ ]:
if ram_records:
    ram_comparison = pd.DataFrame(ram_records)
    print("Ram Comparison (EBVs)")
    print("=" * 60)
    print(ram_comparison.to_string(index=False))
else:
    print("No ram data found. Check your animal IDs above.")

## Checking Inbreeding Coefficient

Before you breed two animals together, it is important to check how related they are. The inbreeding coefficient tells you the probability that offspring will inherit two identical copies of a gene from a common ancestor.

- **0%** means the parents are unrelated.
- **Above 6.25%** is generally considered too high and may cause problems (lower immunity, reduced fertility).

Replace the sire and dam IDs below with animals you are considering mating.

In [ ]:
# Replace with actual sire and dam IDs
sire_id = "SIRE_ID"  # Replace with actual sire ID
dam_id = "DAM_ID"    # Replace with actual dam ID

raw_inbreeding = nsip(
    "inbreeding", "--sire-id", sire_id, "--dam-id", dam_id
)

if raw_inbreeding:
    coeff = raw_inbreeding.get("coefficient", "N/A")
    print(f"Inbreeding coefficient: {coeff}")
    print()
    if isinstance(coeff, (int, float)) and coeff > 0.0625:
        print("WARNING: This mating may produce too much inbreeding.")
        print("Consider choosing a less related sire or dam.")
    else:
        print("This pairing looks acceptable from an inbreeding standpoint.")

## Mating Recommendations

The NSIP system can suggest good mating partners for a specific animal within a breed. This takes into account both genetic merit and relatedness to help you find the best matches.

In [ ]:
# Replace with an actual animal ID from your flock
animal_for_mating = "EXAMPLE_ID"

raw_matings = nsip(
    "matings", "--animal-id", animal_for_mating, "--breed-id", "640"
)

if raw_matings:
    matings_by_rank = pd.DataFrame(raw_matings)
    print(f"Top {min(10, len(matings_by_rank))} recommended mates:")
    print()
    print(matings_by_rank.head(10).to_string(index=False))

## Flock Ranking by Weighted Traits

Not all traits are equally important for every operation. Here we create a simple scoring system that weights the traits you care about most.

The default weights below reflect a typical meat-sheep operation:
- **BWT**: weight of **-1** (negative because *lower* birth weight is better)
- **WWT**: weight of **2** (growth to weaning is very valuable)
- **NLB**: weight of **1.5** (more lambs per ewe = more income)

Adjust these weights to match your own breeding goals.

In [ ]:
# Trait weights -- adjust these to match YOUR priorities
WEIGHTS = {
    "bwt": -1.0,   # Negative: lower birth weight is better
    "wwt": 2.0,    # Positive: higher weaning weight is better
    "nlb": 1.5,    # Positive: more lambs born is better
}

# Search for animals to rank
raw_flock = nsip("search", "--breed-id", "640")

if raw_flock:
    clean_flock = pd.DataFrame(raw_flock)
    print(f"Scoring {len(clean_flock)} animals...")

Now let us calculate a weighted score for each animal and rank them.

In [ ]:
if raw_flock and not clean_flock.empty:
    # Calculate weighted score for each animal
    clean_flock["score"] = 0.0
    for trait, weight in WEIGHTS.items():
        if trait in clean_flock.columns:
            clean_flock["score"] += (
                clean_flock[trait].fillna(0) * weight
            )

    # Rank from best to worst
    flock_by_score = clean_flock.sort_values(
        "score", ascending=False
    )

    print("Top 10 Animals by Weighted Score")
    print("=" * 60)
    display_cols = ["name", "bwt", "wwt", "nlb", "score"]
    available_cols = [c for c in display_cols if c in flock_by_score.columns]
    print(flock_by_score[available_cols].head(10).to_string(index=False))

## MCP Server Approach

Everything we have done with the CLI above can also be done through the NSIP MCP server. The MCP server is useful when:

- You want to ask Claude questions about your sheep in natural language.
- You are building automated workflows that need NSIP data.
- You prefer not to install the CLI binary on every machine.

The Docker container provides the same functionality. Here is how you would call it from Python (equivalent to the CLI approach):

In [ ]:
# Example: Running an NSIP query via Docker instead of the CLI
# This is equivalent to: nsip --json search --breed-id 640 --name storm

# Uncomment to run (requires Docker):
# docker_result = subprocess.run(
#     ["docker", "run", "--rm", "ghcr.io/zircote/nsip:latest",
#      "--json", "search", "--breed-id", "640", "--name", "storm"],
#     capture_output=True, text=True,
# )
# if docker_result.returncode == 0:
#     print(json.dumps(json.loads(docker_result.stdout)[:3], indent=2))

print("When to use each approach:")
print("  CLI:    Quick lookups, scripting, offline use")
print("  MCP:    Claude Desktop integration, team workflows")
print("  Docker: No install needed, consistent environment")

## Conclusion

You now know how to use NSIP data to make informed breeding decisions. Here is the decision workflow:

1. **Search** for candidate animals by breed and name.
2. **Look up** each candidate's EBVs to understand their genetic strengths.
3. **Compare** rams side-by-side to find the best fit for your ewes.
4. **Check inbreeding** before finalizing any mating to avoid problems.
5. **Get recommendations** from NSIP for optimal pairings.
6. **Rank your flock** using weighted scores that match your operation's goals.

### Next Steps

- [ ] Replace example animal IDs with your actual flock IDs and re-run the notebook
- [ ] Adjust the trait weights in the ranking section to match your breeding goals
- [ ] Set up the MCP server in Claude Desktop for conversational flock queries
- [ ] Run inbreeding checks on all planned matings before breeding season
- [ ] Share this notebook with your shepherd or breeding consultant
- [ ] Bookmark the [NSIP website](https://nsip.org) for the latest evaluation runs